# Filter Tiles

---
Filter Sentinel‑2 grid tiles to only those intersecting Indonesia land + shallow coastal waters (from GEBCO bathymetry), inside a land/water/EEZ AOI.

## Overview
**Prereq**: run the [**geomad_tests.ipynb**](GeoMAD_tests.ipynb) notebook first, specifically the save task step to generate the task grids used as the input tile grid here.

1. Load tile grid + boundaries + a supporting raster.
2. Reproject to one CRS for consistent overlays.
3. Clip raster to the AOI to stay in-scope and reduce work.
4. Classify raster cells into “relevant vs not” using a value rule (e.g., depth range).
5. Polygonize the relevant cells and union with fixed polygons (e.g., land) to form one selection footprint.
6. Keep grid tiles that intersect that footprint; then map + report counts/% to verify.

In [ ]:
import geopandas as gpd
import rioxarray
import numpy as np

from rasterio import features
from shapely.geometry import shape

import matplotlib.pyplot as plt

### 1. Load data

**Inputs:**  

- GEBCO bathymetry (raster)  
- Indonesia land (vector)  
- Indonesia land/water/EEZ AOI boundary (vector)  
- Sentinel‑2 grid for the target period (vector)  

In [ ]:
gebco = rioxarray.open_rasterio(
    "/home/jovyan/data/public/bathymetry/gebco/gebco_2025/gebco_2025.tif"
)

land = gpd.read_file(
    "/home/jovyan/data/public/boundaries/indonesia_dissolved_simplified.geojson"
)

land_water_eez = gpd.read_file(
    "/home/jovyan/data/public/boundaries/indonesia_simplified_land_water_aoi.geojson"
)

grid = gpd.read_file("s2_l2a_2024--P1Y-2024--P1Y.geojson")

### 2. Ensure all use the same CRS

Using EPSG:6933 for consistent area-preserving spatial operations and raster/vector alignment.

In [ ]:
target_crs = 6933
land = land.to_crs(target_crs)
land_water_eez = land_water_eez.to_crs(target_crs)
grid = grid.to_crs(target_crs)

gebco = gebco.rio.reproject(6933)

### 3. Quick Visual Check

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

# AOI boundary (land/water/EEZ)
land_water_eez.boundary.plot(ax=ax, color="black", linewidth=1.0, label="AOI boundary")

# Land
land.plot(ax=ax, facecolor="lightgreen", edgecolor="darkgreen", linewidth=0.3, alpha=0.7, label="Land")

# Grid on top
grid.plot(ax=ax, facecolor="none", edgecolor="red", linewidth=0.4, alpha=0.6, label="Tile grid")

ax.set_title("Inputs overview: Land + AOI boundary + Tile grid")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 4. GEBCO: derive shallow-water polygons

Convert GEBCO bathymetry into vector polygons representing shallow water and reef within the AOI.

In [ ]:
# Clip GEBCO to AOI boundary
gebco_clip = gebco.rio.clip(land_water_eez.geometry.values, land_water_eez.crs, drop=True)

In [ ]:
gebco_clip.values[0]

In [ ]:
# Handle nodata to avoid false classifications

nodata = gebco_clip.rio.nodata

# Convert the raster band to a NumPy array of floats
arr = gebco_clip.values[0].astype("float32")

# Replace nodata cells with NaN
arr[arr == nodata] = np.nan

In [ ]:
# Shallow mask
max_depth = -20 # meter
min_depth = 2 # allow a tiny buffer around 0

# create a boolean raster where True is within shallow range
shallow_mask = (arr >= max_depth) & (arr < min_depth)

In [ ]:
# Convert gebco to affine object
transform = gebco_clip.rio.transform()

# Convert the mask raster into vector shapes
shapes_gen = features.shapes(
    shallow_mask.astype(np.uint8),
    mask=shallow_mask,
    transform=transform,
)

# Keep only the shallow polygons as Shapely geometries
shallow_geoms = [shape(geom) for geom, val in shapes_gen if val == 1]

### 5. Build AOI (land + shallow-water polygons)

In [ ]:
combined_geoms = list(land.geometry) + shallow_geoms

# Wraps geometry list into a geodataframe and union all
aoi_union = gpd.GeoSeries(combined_geoms, crs=target_crs).union_all()

In [ ]:
# Quick look on the AOI
fig, ax = plt.subplots(figsize=(8, 8))
gpd.GeoSeries([aoi_union]).plot(ax=ax)
ax.set_title("AOI union")
plt.show()

In [ ]:
# select any tile that touches the AOI
selected = grid[grid.intersects(aoi_union)]

### 6. Inspect Selected Tiles

In [ ]:
# Use Web Mercator
selected_3857 = selected.to_crs(3857)

m = selected_3857.explore(
    color="blue",
    style_kwds={"fillOpacity": 0.2, "weight": 1, "color": "blue"},
    tiles="Esri.WorldImagery"
)
m

### 7. Summary stats

In [ ]:
total = len(grid)
sel = len(selected)

print(f"Grid tiles (total):     {total:,}")
print(f"Selected tiles:         {sel:,}")
print(f"Not selected tiles:     {total - sel:,}")
print(f"Selected % of grid:     {sel/total:.2%}")